Chat GPT code!

In [14]:
import cv2
import json
import os
import re

# Specify the folder containing videos and output directories
video_folder = "C:\\awilde\\britta\\LTU\\SkiingProject\\SkiersProject\\Data\\test\\"
output_frame_path = "C:\\awilde\\britta\\LTU\\SkiingProject\\SkiersProject\\Data\\Frames_of_videos\\"
annotation_folder = "C:\\awilde\\britta\LTU\\SkiingProject\\SkiersProject\\Data\\Annotations\\"  # Folder with corresponding JSON files
combined_json_path = "C:\\awilde\\britta\LTU\\SkiingProject\\SkiersProject\\Data\\Annotations\\combined_annotations.json"

# Maximum frame limits for specific videos (optional)
frame_limits = {
    "DJI_0009.MP4": 890
}

# Initialize counters
frame_counter = 0
combined_data = {
    "licenses" : [],
    "info" : [],
    "images": [],
    "annotations": [],
    "categories": []
}

In [15]:
# Function to extract the corresponding JSON filename
def get_json_filename(video_name):
    match = re.search(r'DJI_0*(\d+)(_cut)?', video_name)
    if match:
        number = match.group(1).zfill(2)  # Extract the numeric part
        suffix = match.group(2) if match.group(2) else ""  # Extract '_cut' if present
        return f"{number}{suffix}.json"
    return None  # Return None if no match is found

In [16]:
image_id_counter = 1  # Start image IDs at 1
annotation_id_counter = 1  # Start annotation IDs at 1

video_files = [f for f in os.listdir(video_folder) if f.endswith(('.mp4', '.MP4'))]

# Process each video and corresponding JSON
for video_file in video_files:
    video_path = os.path.join(video_folder, video_file)
    json_file_name = get_json_filename(video_file)
    print(json_file_name)
    #json_path = os.path.join(annotation_folder, f"{os.path.splitext(video_file)[0]}_annotations.json")
    json_path = os.path.join(annotation_folder, json_file_name)
    print(json_path)

    # Check if the JSON file exists
    if not os.path.exists(json_path):
        print(f"Warning: JSON file not found for {video_file}. Skipping...")
        continue

    # Split video into frames
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    current_frame = 0
    max_frames = frame_limits.get(video_file, float('inf'))  # Default to no limit if not specified

    while success and current_frame <= max_frames:
        # Save frame with sequential numbering based on global frame_counter
        frame_name = f"frame_{frame_counter:06d}.jpg"
        save_path = os.path.join(output_frame_path, frame_name)
        cv2.imwrite(save_path, image)

        # Add to combined JSON
        combined_data["images"].append({
            "id": image_id_counter,
            "width": image.shape[1],
            "height": image.shape[0],
            "file_name": frame_name,
            "license": 0,
            "flickr_url": "",
            "coco_url": "",
            "date_captured": 0
        })

        success, image = vidcap.read()
        frame_counter += 1
        image_id_counter += 1  # Increment image ID counter
        current_frame += 1

    # Update JSON file
    with open(json_path, 'r') as f:
        data = json.load(f)

    for annotation in data.get("annotations", []):
        # Assign a sequential ID and link to the correct image
        annotation["id"] = annotation_id_counter
        annotation["image_id"] = annotation_id_counter  # Align with the correct frame ID
        combined_data["annotations"].append(annotation)
        annotation_id_counter += 1

    # Add categories only once (assumes they are consistent across files)
    if not combined_data["categories"]:
        combined_data["categories"] = data.get("categories", [])

# Save the combined JSON
with open(combined_json_path, 'w') as f:
    json.dump(combined_data, f, indent=4)

print(f"Processed {frame_counter} frames in total.")

09.json
C:\awilde\britta\LTU\SkiingProject\SkiersProject\Data\Annotations\09.json
14_cut.json
C:\awilde\britta\LTU\SkiingProject\SkiersProject\Data\Annotations\14_cut.json
15_cut.json
C:\awilde\britta\LTU\SkiingProject\SkiersProject\Data\Annotations\15_cut.json
17_cut.json
C:\awilde\britta\LTU\SkiingProject\SkiersProject\Data\Annotations\17_cut.json
18_cut.json
C:\awilde\britta\LTU\SkiingProject\SkiersProject\Data\Annotations\18_cut.json
22_cut.json
C:\awilde\britta\LTU\SkiingProject\SkiersProject\Data\Annotations\22_cut.json
25.json
C:\awilde\britta\LTU\SkiingProject\SkiersProject\Data\Annotations\25.json
38.json
C:\awilde\britta\LTU\SkiingProject\SkiersProject\Data\Annotations\38.json
40.json
C:\awilde\britta\LTU\SkiingProject\SkiersProject\Data\Annotations\40.json
43.json
C:\awilde\britta\LTU\SkiingProject\SkiersProject\Data\Annotations\43.json
44.json
C:\awilde\britta\LTU\SkiingProject\SkiersProject\Data\Annotations\44.json
54_cut.json
C:\awilde\britta\LTU\SkiingProject\SkiersProje

# Verification

In [17]:
import random
import cv2

# Draw annotations on a frame
def draw_annotations(frame, annotations, categories):
    for ann in annotations:
        keypoints = ann.get("keypoints", [])
        category_id = ann.get("category_id", None)
        category_name = next((cat["name"] for cat in categories if cat["id"] == category_id), "unknown")
        
        # Draw keypoints
        for i in range(0, len(keypoints), 3):
            x, y, v = keypoints[i:i+3]  # x, y, visibility
            if v > 0:  # Only draw visible keypoints
                cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)  # Green keypoints

        # Optionally, draw lines (skeleton) if available
        skeleton = categories[0].get("skeleton", [])
        for connection in skeleton:
            start_idx, end_idx = connection
            if start_idx < len(keypoints) // 3 and end_idx < len(keypoints) // 3:
                x1, y1, v1 = keypoints[start_idx * 3:start_idx * 3 + 3]
                x2, y2, v2 = keypoints[end_idx * 3:end_idx * 3 + 3]
                if v1 > 0 and v2 > 0:  # Only draw lines between visible points
                    cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)  # Blue lines

    return frame

# Verify frames by drawing annotations
def verify_frames_with_annotations(images, annotations, frame_path, categories, num_verifications=5):
    print("\n--- Verifying Frames with Annotations ---")
    for _ in range(num_verifications):
        # Pick a random image
        image = random.choice(images)
        frame_file = os.path.join(frame_path, image["file_name"])

        # Find associated annotations
        associated_annotations = [ann for ann in annotations if ann["image_id"] == image["id"]]

        print(f"Verifying Frame: {image['file_name']}, ID: {image['id']}")
        print("Annotations:")
        for ann in associated_annotations:
            print(f"  - ID: {ann['id']}, Keypoints: {ann['keypoints']}")

        # Load and annotate the frame
        frame = cv2.imread(frame_file)
        if frame is not None:
            annotated_frame = draw_annotations(frame, associated_annotations, categories)

            # Display the annotated frame
            cv2.imshow(f"Frame {image['id']} with Annotations", annotated_frame)
            cv2.waitKey(0)  # Press any key to close the frame window
            cv2.destroyAllWindows()
        else:
            print(f"Warning: Unable to load frame {frame_file}")

# Call verification after processing all videos
verify_frames_with_annotations(
    combined_data["images"], 
    combined_data["annotations"], 
    output_frame_path, 
    combined_data["categories"]
)


--- Verifying Frames with Annotations ---
Verifying Frame: frame_006212.jpg, ID: 6213
Annotations:
  - ID: 6213, Keypoints: [1046.16, 565.54, 1, 1049.73, 561.97, 1, 1042.6, 561.97, 1, 1055.07, 558.41, 1, 1037.25, 558.41, 1, 1066.35, 566.94, 1, 1028.76, 567.71, 1, 1078.92, 577.99, 1, 1014.25, 580.2, 1, 1081.25, 606.39, 1, 1006.6, 603.54, 1, 1062.2, 613.66, 1, 1039.03, 617.23, 1, 1083.59, 651.09, 1, 1040.82, 660.0, 1, 1104.98, 677.82, 1, 1044.38, 695.65, 1, 1046.16, 547.72, 1, 1046.16, 567.32, 1, 1049.73, 613.66, 1, 1117.46, 692.08, 1, 1043.98, 710.71, 1, 1119.24, 686.74, 1, 1040.03, 708.52, 1, 1103.2, 681.39, 1, 1047.95, 697.43, 1]
Verifying Frame: frame_007709.jpg, ID: 7710
Annotations:
  - ID: 7710, Keypoints: [848.79, 575.99, 1, 850.45, 571.01, 1, 848.79, 571.01, 1, 860.41, 572.67, 1, 860.41, 572.67, 1, 869.73, 583.85, 1, 872.33, 577.47, 1, 884.25, 611.95, 1, 880.09, 603.41, 1, 881.99, 638.12, 1, 862.17, 603.51, 1, 905.72, 622.56, 1, 901.91, 620.8, 1, 884.47, 659.86, 1, 885.05, 655.